In [2]:
!pip install tensorflow pandas matplotlib sklearn 

In [3]:
!pip list

Package                       Version
----------------------------- --------------------
absl-py                       1.4.0
aiofiles                      23.1.0
aiohttp                       3.8.4
aiosignal                     1.3.1
alabaster                     0.7.12
altair                        4.2.2
anaconda-client               1.11.0
anaconda-navigator            2.3.1
anaconda-project              0.11.1
anyio                         3.5.0
appdirs                       1.4.4
argon2-cffi                   21.3.0
argon2-cffi-bindings          21.2.0
arrow                         1.2.2
astroid                       2.11.7
astropy                       5.1
astunparse                    1.6.3
async-timeout                 4.0.2
atomicwrites                  1.4.0
attrs                         21.4.0
Automat                       20.2.0
autopep8                      1.6.0
Babel                         2.9.1
backcall                      0.2.0
backports.functools-lru-cache 1.6.4
back

In [4]:
import os
import tensorflow as tf
import numpy as np
import pandas as pd

In [5]:
os.path.join('Kaggle_dataset_toxic_comments','train.csv','train.csv')

'Kaggle_dataset_toxic_comments\\train.csv\\train.csv'

In [6]:
df=pd.read_csv(os.path.join('Kaggle_dataset_toxic_comments','train.csv','train.csv'))

In [7]:
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [8]:
from tensorflow.keras.layers import TextVectorization

In [9]:
x=df['comment_text']
y=df[df.columns[2:]].values

In [10]:
MAX_WORDS=20000  

In [11]:
vector=TextVectorization(max_tokens=MAX_WORDS,output_sequence_length=2000,output_mode='int')

In [12]:
vector.adapt(x.values)

In [13]:
vector("ohh jure !")[:2]

<tf.Tensor: shape=(2,), dtype=int64, numpy=array([19312, 19425], dtype=int64)>

In [14]:
v=vector(x.values)

In [15]:
data=tf.data.Dataset.from_tensor_slices((v,y))   # v cest le input / y le résultat output
data=data.cache()
data=data.shuffle(160000)


data=data.batch(16)
data=data.prefetch(8)

In [16]:
batch_x,batch_y=data.as_numpy_iterator().next()

In [17]:
len(data)

9974

In [18]:
train=data.take(int(len(data)*.7))
valid=data.skip(int(len(data)*.7)).take(int(len(data)*.2))
test=data.skip(int(len(data)*.7)+int(len(data)*.2)).take(int(len(data)*.1))

In [19]:
len(test)

997

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Bidirectional, Dropout,Dense,Embedding

In [21]:
model=Sequential()

In [22]:
model.add(tf.keras.layers.Embedding(MAX_WORDS+1,32))
model.add(tf.keras.layers.Bidirectional(LSTM(32)))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(256,activation='relu'))
model.add(tf.keras.layers.Dense(128,activation='relu'))
model.add(tf.keras.layers.Dense(6,activation='sigmoid'))    #on a 6 differents résultats:toxic/severe_toxic/obscene...


In [23]:
model.compile(loss='BinaryCrossentropy',optimizer='Adam',metrics=['accuracy'])

In [24]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 32)          640032    
                                                                 
 bidirectional (Bidirectiona  (None, 64)               16640     
 l)                                                              
                                                                 
 dense (Dense)               (None, 256)               16640     
                                                                 
 dense_1 (Dense)             (None, 256)               65792     
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 6)                 774       
                                                        

In [25]:
model.fit(train,epochs=1,validation_data=valid)

6981/6981 [==============================] - 3672s 526ms/step - loss: 0.0628 - accuracy: 0.9858 - val_loss: 0.0475 - val_accuracy: 0.9941


In [26]:
example_predict=vector("i will kill you!")

In [96]:
vector("i will kill you!")

<tf.Tensor: shape=(2000,), dtype=int64, numpy=array([  8,  44, 950, ...,   0,   0,   0], dtype=int64)>

In [27]:
predict=model.predict(np.expand_dims(example_predict,0))

1/1 [==============================] - 0s 473ms/step


In [28]:
predict

array([[0.66167957, 0.01636995, 0.20132545, 0.02272746, 0.23219235,
        0.05881525]], dtype=float32)

In [78]:
!pip install Gradio jinja2

In [79]:
import tensorflow as tf
import gradio as gr

In [100]:
def comment(comment):
    vet=vector([comment])
    print(len(vet))
    result=model.predict(vet)
    text=''
    for idx , col in enumerate(df.columns[2:-1]):
        text+='{}:{} \n'.format(col,result[0][idx]>0.5)
        
        
    return text    

In [101]:
interface=gr.Interface(fn=comment,inputs=gr.inputs.Textbox(lines=2,placeholder='Comment '),outputs='text')

In [104]:
interface.launch()


Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Running on local URL:  http://127.0.0.1:7868

To create a public link, set `share=True` in `launch()`.
